<a href="https://colab.research.google.com/github/Vishy-A/MachineLearningIntro/blob/main/HW5/4105HW5P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
#Vishy Adusumilli
#801185663
#HW 5 Problems 3
import numpy as np
import pandas as pd
import seaborn as sns;
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems = 2, linewidth = 75)
import matplotlib.pyplot as plt
from google.colab import drive
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Machine Learning/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [115]:
varsall = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus', 'price']
varstrings = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
def binarymap(inp):
  return inp.map({'yes' : 1, 'no' : 0, 'furnished' : 2, 'semi-furnished' : 1, 'unfurnished' : 0 })

dataset[varstrings] = dataset[varstrings].apply(binarymap)


dataset.head()


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


In [116]:
standardScaler = StandardScaler()

dataset[varsall] = standardScaler.fit_transform(dataset[varsall])
np.random.seed(0)
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4.566365,1.046726,1.403419,1.421812,1.378217,0.405623,-0.465315,-0.734539,-0.219265,1.472618,1.517692,1.804941,1.406286
1,4.004484,1.757010,1.403419,5.405809,2.532024,0.405623,-0.465315,-0.734539,-0.219265,1.472618,2.679409,-0.554035,1.406286
2,4.004484,2.218232,0.047278,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,-0.679063,1.517692,1.804941,0.091662
3,3.985755,1.083624,1.403419,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,1.472618,2.679409,1.804941,1.406286
4,3.554979,1.046726,1.403419,-0.570187,0.224410,0.405623,2.149083,1.361397,-0.219265,1.472618,1.517692,-0.554035,1.406286


In [117]:
x = dataset[varsall].values
y = dataset.pop('price')

x = torch.tensor(x)
y = torch.tensor(y)

In [118]:
samples = x.shape[0]
n_val = int(.2 * samples)

shuffledindices = torch.randperm(samples)

trainindices = shuffledindices[:-n_val]
valindices = shuffledindices[-n_val:]

trainindices, valindices



(tensor([ 58, 258, 243,  64, 232,  45,  10, 166, 398, 281, 485, 449, 486,
          21,  49, 294,  12, 198, 527, 200,  42,  67,  95, 473, 118, 159,
         507, 216,  81, 147, 395, 439, 240,  22, 481, 128, 185, 139,  11,
         236, 393, 188, 293,  82, 256, 130, 331, 300, 172, 266, 489, 196,
         462, 238, 312, 444,  61, 382, 347, 257, 263, 464, 217, 160, 400,
         168,  78, 265, 169, 295, 297,  13, 242,  23, 529, 337, 371,  41,
           2, 443, 288, 141, 342, 106, 461,  51, 235, 366, 430, 466, 440,
          25, 424, 358, 177, 264, 157, 533, 419, 392, 149, 132,  98,  53,
         162, 407, 509,   4, 121, 524, 387, 276, 493, 420, 349, 403, 248,
         327, 252, 380, 179, 532,  48, 180,   7,  66, 136, 495, 244, 309,
          94, 354, 170, 518, 282, 318, 525, 437,  96, 415, 153, 465, 137,
         296, 304,  55, 506, 369,  34,   6, 233, 167, 289,  20, 417, 219,
          79, 405, 175, 326,  35, 191, 103, 365, 100, 537,  52, 441, 428,
         161, 152, 270,  60, 181,  32,

In [119]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]

xntrain = .1 * xtrain
xnval = .1 * valx

In [120]:
def model(x, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, b):
  return w1 * x[:,0] + w2 * x[:,1] + w3 * x[:,2] + w4 * x[:,3] + w5 * x[:,4] + w6 * x[:,5] + w7 * x[:,6] + w8 * x[:,7] + w9 * x[:,8] + w10 * x[:,9] + w11 * x[:,10]+ b

In [121]:
def loss_func(t_p,t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [122]:
def training(epochs, optimizer, params, xtrain, ytrain, xtest, ytest):
  for epoch in range(1, epochs + 1):
    t_p = model(xtrain, *params)
    loss = loss_func(t_p, ytrain)

    v_p = model(xtest, *params)
    vloss = loss_func(v_p, ytest)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



    if epoch % 500 == 0:
      print('Epoch %d, Loss %f, Validation Loss %g' % (epoch, float(loss), float(vloss)))


  return params

In [123]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .1
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.334037, Validation Loss 0.309703
Epoch 1000, Loss 0.334037, Validation Loss 0.309703
Epoch 1500, Loss 0.334037, Validation Loss 0.309703
Epoch 2000, Loss 0.334037, Validation Loss 0.309703
Epoch 2500, Loss 0.334037, Validation Loss 0.309703
Epoch 3000, Loss 0.334037, Validation Loss 0.309703
Epoch 3500, Loss 0.334037, Validation Loss 0.309703
Epoch 4000, Loss 0.334037, Validation Loss 0.309703
Epoch 4500, Loss 0.334037, Validation Loss 0.309703
Epoch 5000, Loss 0.334037, Validation Loss 0.309703


tensor([ 0.2808,  0.0533,  0.2583,  0.2126,  0.0757,  0.0552,  0.0653,
         0.0992,  0.2257,  0.1696,  0.1618, -0.0035], requires_grad=True)

In [124]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .01
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.334038, Validation Loss 0.309772
Epoch 1000, Loss 0.334037, Validation Loss 0.309703
Epoch 1500, Loss 0.334037, Validation Loss 0.309703
Epoch 2000, Loss 0.334037, Validation Loss 0.309703
Epoch 2500, Loss 0.334037, Validation Loss 0.309703
Epoch 3000, Loss 0.334037, Validation Loss 0.309703
Epoch 3500, Loss 0.334037, Validation Loss 0.309703
Epoch 4000, Loss 0.334037, Validation Loss 0.309703
Epoch 4500, Loss 0.334037, Validation Loss 0.309703
Epoch 5000, Loss 0.334037, Validation Loss 0.309703


tensor([ 0.2808,  0.0533,  0.2583,  0.2126,  0.0757,  0.0552,  0.0653,
         0.0992,  0.2257,  0.1696,  0.1618, -0.0035], requires_grad=True)

In [125]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .001
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.618014, Validation Loss 0.456044
Epoch 1000, Loss 0.357201, Validation Loss 0.316409
Epoch 1500, Loss 0.337985, Validation Loss 0.312419
Epoch 2000, Loss 0.334865, Validation Loss 0.311554
Epoch 2500, Loss 0.334231, Validation Loss 0.310888
Epoch 3000, Loss 0.334086, Validation Loss 0.310413
Epoch 3500, Loss 0.334050, Validation Loss 0.310112
Epoch 4000, Loss 0.334041, Validation Loss 0.309933
Epoch 4500, Loss 0.334038, Validation Loss 0.309831
Epoch 5000, Loss 0.334038, Validation Loss 0.309774


tensor([ 0.2806,  0.0534,  0.2579,  0.2129,  0.0758,  0.0551,  0.0655,
         0.0992,  0.2257,  0.1699,  0.1618, -0.0034], requires_grad=True)

In [126]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .0001
optimizer = optim.SGD([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 11.511718, Validation Loss 8.85604
Epoch 1000, Loss 7.413383, Validation Loss 5.62564
Epoch 1500, Loss 4.852615, Validation Loss 3.62728
Epoch 2000, Loss 3.245656, Validation Loss 2.38867
Epoch 2500, Loss 2.231777, Validation Loss 1.61911
Epoch 3000, Loss 1.587785, Validation Loss 1.13951
Epoch 3500, Loss 1.175348, Validation Loss 0.839502
Epoch 4000, Loss 0.908556, Validation Loss 0.650974
Epoch 4500, Loss 0.733911, Validation Loss 0.531847
Epoch 5000, Loss 0.617987, Validation Loss 0.456079


tensor([ 0.2300,  0.1473,  0.2093,  0.3078,  0.2396,  0.1784,  0.2525,
         0.3905,  0.3060,  0.2686,  0.2717, -0.0062], requires_grad=True)

In [127]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .1
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.334037, Validation Loss 0.309703
Epoch 1000, Loss 0.334037, Validation Loss 0.309703
Epoch 1500, Loss 0.334038, Validation Loss 0.309689
Epoch 2000, Loss 0.334037, Validation Loss 0.309703
Epoch 2500, Loss 0.334037, Validation Loss 0.309703
Epoch 3000, Loss 0.334037, Validation Loss 0.309705
Epoch 3500, Loss 0.334037, Validation Loss 0.309703
Epoch 4000, Loss 0.334043, Validation Loss 0.30962
Epoch 4500, Loss 0.334037, Validation Loss 0.309703
Epoch 5000, Loss 0.334037, Validation Loss 0.309703


tensor([ 0.2808,  0.0533,  0.2583,  0.2126,  0.0757,  0.0552,  0.0653,
         0.0992,  0.2257,  0.1696,  0.1618, -0.0035], requires_grad=True)

In [128]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .01
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 0.334054, Validation Loss 0.310173
Epoch 1000, Loss 0.334037, Validation Loss 0.309703
Epoch 1500, Loss 0.334037, Validation Loss 0.309703
Epoch 2000, Loss 0.334037, Validation Loss 0.309703
Epoch 2500, Loss 0.334037, Validation Loss 0.309703
Epoch 3000, Loss 0.334037, Validation Loss 0.309703
Epoch 3500, Loss 0.334037, Validation Loss 0.309703
Epoch 4000, Loss 0.334037, Validation Loss 0.309703
Epoch 4500, Loss 0.334037, Validation Loss 0.309706
Epoch 5000, Loss 0.334037, Validation Loss 0.309703


tensor([ 0.2808,  0.0533,  0.2583,  0.2126,  0.0757,  0.0552,  0.0653,
         0.0992,  0.2257,  0.1696,  0.1618, -0.0035], requires_grad=True)

In [129]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .001
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 4.701329, Validation Loss 3.58907
Epoch 1000, Loss 0.963523, Validation Loss 0.722318
Epoch 1500, Loss 0.388353, Validation Loss 0.321979
Epoch 2000, Loss 0.340870, Validation Loss 0.308115
Epoch 2500, Loss 0.335670, Validation Loss 0.311428
Epoch 3000, Loss 0.334392, Validation Loss 0.311358
Epoch 3500, Loss 0.334094, Validation Loss 0.310513
Epoch 4000, Loss 0.334043, Validation Loss 0.309968
Epoch 4500, Loss 0.334038, Validation Loss 0.309762
Epoch 5000, Loss 0.334037, Validation Loss 0.309712


tensor([ 0.2808,  0.0534,  0.2583,  0.2125,  0.0757,  0.0552,  0.0652,
         0.0992,  0.2257,  0.1696,  0.1618, -0.0035], requires_grad=True)

In [130]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learningrate = .0001
optimizer = optim.Adam([params], lr = learningrate)

training(
    epochs = 5000,
    optimizer = optimizer,
    params = params,
    xtrain = xtrain,
    ytrain = ytrain,
    xtest = valx,
    ytest = valy
    )

Epoch 500, Loss 16.067233, Validation Loss 12.4991
Epoch 1000, Loss 14.224165, Validation Loss 11.049
Epoch 1500, Loss 12.535595, Validation Loss 9.72114
Epoch 2000, Loss 10.989023, Validation Loss 8.50617
Epoch 2500, Loss 9.574300, Validation Loss 7.39609
Epoch 3000, Loss 8.283245, Validation Loss 6.38438
Epoch 3500, Loss 7.109257, Validation Loss 5.46578
Epoch 4000, Loss 6.046944, Validation Loss 4.63601
Epoch 4500, Loss 5.091776, Validation Loss 3.89145
Epoch 5000, Loss 4.239777, Validation Loss 3.22891


tensor([ 0.5542,  0.5496,  0.5534,  0.5532,  0.5492,  0.5472,  0.5457,
         0.5465,  0.5540,  0.5521,  0.5505, -0.0154], requires_grad=True)